# ALBERT pretraining on Japanese wikipedia corpus (v2)

This notebook is assumed to be executed on Colaboratory notebook with TPU.



In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf

In [3]:
tf.__version__

'1.15.2'

In [4]:
!git clone --recursive https://github.com/alinear-corp/albert-japanese.git

Cloning into 'albert-japanese'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 256 (delta 10), reused 20 (delta 8), pack-reused 230
Receiving objects: 100% (256/256), 384.37 KiB | 3.84 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Submodule 'ALBERT' (https://github.com/alinear-corp/ALBERT.git) registered for path 'ALBERT'
Submodule 'wikiextractor' (https://github.com/attardi/wikiextractor.git) registered for path 'wikiextractor'
Cloning into '/content/albert-japanese/ALBERT'...
remote: Enumerating objects: 11, done.        
remote: Counting objects: 100% (11/11), done.        
remote: Compressing objects: 100% (9/9), done.        
remote: Total 356 (delta 3), reused 5 (delta 2), pack-reused 345        
Receiving objects: 100% (356/356), 248.44 KiB | 4.69 MiB/s, done.
Resolving deltas: 100% (231/231), done.
Cloning into '/content/albert-japanese/wikiextractor'...
remote: Enumera

In [5]:
cd albert-japanese/ALBERT

/content/albert-japanese/ALBERT


In [6]:
!git log --oneline | head

0cd5fb8 Change module name
9fe8be1 Keep accents for Japanese
e198b4c Merge remote-tracking branch 'google/master'
a41cf11 Fix instructions on running spm_train. Use the right " character.
520bc03 rename the tutorial and add a link to open it from colab.
d99e15e ALBERT GLUE fine-tuning colab tutorial.
557c37b fix an error on the max_seq_length. Here we should set it to 512 instead of 128.
4135001 Handle cases where SP can't load the file, but gfile can.
d246aaf Merge pull request #163 from arrrrrmin/master
210263e Merge pull request #186 from twilightdema/hotfix/Fix_RACE_error_no_all.txt


Authentication to use TPU.

In [7]:
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Check TPU devices

In [8]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.77.60.186:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 13029326126212796012),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4459528194544362541),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13688982376942381197),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7687019565568627200),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 13370544531958382221),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 17184390719206556035),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9400966331743721856),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15063699715366469011),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 17650163070

## Set input and output

Need to put `all-maxseq(128|512).tfrecord` data for pre-traning on your GCS bucket.  
Trained objects will be saved into a specified GCS bucket.

In [ ]:
INPUT_DATA_GCS = 'gs://albert_japanese_v2/wiki_tfrecord'

In [ ]:
TARGET_DIRS = [
  'AA',
  'AB',
  'AC',
  'AD',
  'AE',
  'AF',
  'AG',
  'AH',
  'AI',
  'AJ',
  'AK',
  'AL',
  'AM',
  'AN',
  'AO',
  'AP',
  'AQ',
  'AR',
  'AS',
  'AT',
  'AU',
  'AV',
  'AW',
  'AX',
  'AY',
  'AZ',
  'BA',
  'BB',
  'BC',
]

In [ ]:
# MAX_SEQ_LEN = 128
MAX_SEQ_LEN = 512

In [ ]:
INPUT_FILE = ','.join( [ '{}/{}/all-maxseq{}.tfrecord'.format(INPUT_DATA_GCS, elem, MAX_SEQ_LEN) for elem in TARGET_DIRS] )

In [ ]:
OUTPUT_GCS = 'gs://albert_japanese_v2/model'

## Execute pre-training



In [14]:
cd /content/

/content


In [ ]:
# !PYTHONPATH=/tensorflow-1.15.2/python3.6:/content/albert-japanese/ python albert-japanese/ALBERT/run_pretraining.py \
#   --albert_config_file=albert-japanese/albert_config.json \
#   --input_file={INPUT_FILE} \
#   --output_dir={OUTPUT_GCS} \
#   --use_tpu=True \
#   --tpu_name={TPU_ADDRESS} \
#   --num_tpu_cores=8 \
#   --do_train=True \
#   --do_eval=True \
#   --train_batch_size=256 \
#   --max_seq_length={MAX_SEQ_LEN} \
#   --max_predictions_per_seq=20 \
#   --num_train_steps=1000000 \
#   --num_warmup_steps=10000 \
#   --save_checkpoints_steps=10000 \
#   --learning_rate=1e-4

INFO:tensorflow:*** Input Files ***
I0421 03:58:36.453987 140056390174592 run_pretraining.py:484] *** Input Files ***
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AA/all-maxseq128.tfrecord
I0421 03:58:36.454255 140056390174592 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AA/all-maxseq128.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AB/all-maxseq128.tfrecord
I0421 03:58:36.454352 140056390174592 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AB/all-maxseq128.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AC/all-maxseq128.tfrecord
I0421 03:58:36.454436 140056390174592 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AC/all-maxseq128.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AD/all-maxseq128.tfrecord
I0421 03:58:36.454512 140056390174592 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AD/all-maxseq128.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/

In [15]:
!PYTHONPATH=/tensorflow-1.15.2/python3.6:/content/albert-japanese/ python albert-japanese/ALBERT/run_pretraining.py \
  --albert_config_file=albert-japanese/albert_config.json \
  --input_file={INPUT_FILE} \
  --output_dir={OUTPUT_GCS} \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --num_tpu_cores=8 \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length={MAX_SEQ_LEN} \
  --max_predictions_per_seq=20 \
  --num_train_steps=1400000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4

INFO:tensorflow:*** Input Files ***
I0424 05:46:27.638490 140579898521472 run_pretraining.py:484] *** Input Files ***
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AA/all-maxseq512.tfrecord
I0424 05:46:27.638835 140579898521472 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AA/all-maxseq512.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AB/all-maxseq512.tfrecord
I0424 05:46:27.638947 140579898521472 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AB/all-maxseq512.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AC/all-maxseq512.tfrecord
I0424 05:46:27.639044 140579898521472 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AC/all-maxseq512.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/wiki_tfrecord/AD/all-maxseq512.tfrecord
I0424 05:46:27.639133 140579898521472 run_pretraining.py:486]   gs://albert_japanese_v2/wiki_tfrecord/AD/all-maxseq512.tfrecord
INFO:tensorflow:  gs://albert_japanese_v2/